棋盘坐标 E4, 转化为坐标形式就是 (3, 4), 坐标数值大小是从 0 开始，到 7 结束。  

Board 类中，提供以上两种坐标的转化方法：
+ `board_num(action)`: 棋盘坐标转化为数字坐标。
    + action: 棋盘坐标，e.g. 'G6'
    + 返回值: 数字坐标，e.g. (5, 6)
+ `num_board(action)`: 数字坐标转化为棋盘坐标。
    + action: 数字坐标，e.g. (2, 7)
    + 返回值: 棋盘坐标，e.g. 'H3'

# 2.2 创建随机玩家

In [1]:
# 导入随机包
import random

class RandomPlayer:
    """
    随机玩家, 随机返回一个合法落子位置
    """

    def __init__(self, color):
        """
        玩家初始化
        :param color: 下棋方，'X' - 黑棋，'O' - 白棋
        """
        self.color = color
        

    def random_choice(self, board):
        """
        从合法落子位置中随机选一个落子位置
        :param board: 棋盘
        :return: 随机合法落子位置, e.g. 'A1' 
        """
        # 用 list() 方法获取所有合法落子位置坐标列表
        action_list = list(board.get_legal_actions(self.color))

        # 如果 action_list 为空，则返回 None,否则从中选取一个随机元素，即合法的落子坐标
        if len(action_list) == 0:
            return None
        else:
            return random.choice(action_list)

    def get_move(self, board):
        """
        根据当前棋盘状态获取最佳落子位置
        :param board: 棋盘
        :return: action 最佳落子位置, e.g. 'A1'
        """
        if self.color == 'X':
            player_name = '黑棋'
        else:
            player_name = '白棋'
        print("请等一会，对方 {}-{} 正在思考中...".format(player_name, self.color))
        action = self.random_choice(board)
        return action

# 2.3 创建人类

In [2]:
class HumanPlayer:
    """
    人类玩家
    """

    def __init__(self, color):
        """
        玩家初始化
        :param color: 下棋方，'X' - 黑棋，'O' - 白棋
        """
        self.color = color
    

    def get_move(self, board):
        """
        根据当前棋盘输入人类合法落子位置
        :param board: 棋盘
        :return: 人类下棋落子位置
        """
        # 如果 self.color 是黑棋 "X",则 player 是 "黑棋"，否则是 "白棋"
        if self.color == "X":
            player = "黑棋"
        else:
            player = "白棋"

        # 人类玩家输入落子位置，如果输入 'Q', 则返回 'Q'并结束比赛。
        # 如果人类玩家输入棋盘位置，e.g. 'A1'，
        # 首先判断输入是否正确，然后再判断是否符合黑白棋规则的落子位置
        while True:
            action = input(
                    "请'{}-{}'方输入一个合法的坐标(e.g. 'D3'，若不想进行，请务必输入'Q'结束游戏。): ".format(player,
                                                                                 self.color))

            # 如果人类玩家输入 Q 则表示想结束比赛
            if action == "Q" or action == 'q':
                return "Q"
            else:
                row, col = action[1].upper(), action[0].upper()

                # 检查人类输入是否正确
                if row in '12345678' and col in 'ABCDEFGH':
                    # 检查人类输入是否为符合规则的可落子位置
                    if action in board.get_legal_actions(self.color):
                        return action
                else:
                    print("你的输入不合法，请重新输入!")


# 2.3 创建AI

In [3]:
import copy

class AIPlayerMiniMax:
    """
    AI 玩家
    """

    def __init__(self, color):
        """
        玩家初始化
        :param color: 下棋方，'X' - 黑棋，'O' - 白棋
        """

        self.color = color
        
        self.INFINITY = 100
        if color == 'X':
            self.color_oppo = 'O'
        else:
            self.color_oppo = 'X'
        
        
    def game_over(self,board):
        b_list = list(board.get_legal_actions('X'))
        w_list = list(board.get_legal_actions('O'))
        is_over =  len(b_list) == 0 and len(w_list) == 0
        return is_over 
    
    def minimax_decision(self, board):
        best_value = 0
        best_action = None
        
        for action in list(board.get_legal_actions(self.color)):
            temp_board = copy.deepcopy(board)
            temp_board._move(action, self.color)      
            temp_value = self.min_value(temp_board)
            
            if temp_value > best_value:
                best_action = action
                best_value = temp_value
        return best_action
        
    def max_value(self, board):
        if self.game_over(board):
          
            winner, temp_value = board.get_winner()
            if (winner == 0 and self.color == 'X') or (winner == 1 and self.color == 'O'):
                return temp_value
            else:
                return -temp_value
        
        value = -self.INFINITY
        for action in list(board.get_legal_actions(self.color)):
            temp_board = copy.deepcopy(board)
            temp_board._move(action, self.color)
            
            temp_value = self.min_value(temp_board)
            value = max(value, temp_value)
        
        if len(list(board.get_legal_actions(self.color))) == 0:
            temp_board = copy.deepcopy(board)
            value = self.min_value(temp_board)
        
        return value
        
        
    def min_value(self, board):
        if self.game_over(board):
          
            winner, temp_value = board.get_winner()
            if (winner == 0 and self.color == 'X') or (winner == 1 and self.color == 'O'):
                return temp_value
            else:
                return -temp_value
        
        value = self.INFINITY
        for action in list(board.get_legal_actions(self.color_oppo)):
            temp_board = copy.deepcopy(board)
            temp_board._move(action, self.color_oppo)
            
            temp_value = self.max_value(temp_board)
            value = min(value, temp_value)
            
        if len(list(board.get_legal_actions(self.color_oppo))) == 0:
            temp_board = copy.deepcopy(board)
            value = self.max_value(temp_board)
                
        return value
        
        

    def get_move(self, board):
        """
        根据当前棋盘状态获取最佳落子位置
        :param board: 棋盘
        :return: action 最佳落子位置, e.g. 'A1'
        """
        if self.color == 'X':
            player_name = '黑棋'
        else:
            player_name = '白棋'
        print("请等一会，对方 {}-{} 正在思考中...".format(player_name, self.color))

        # -----------------请实现你的算法代码--------------------------------------
        action = self.minimax_decision(board)
        # ------------------------------------------------------------------------

        return action


## 2 alpha beta

In [65]:
import copy

class AIPlayer0:
    """
    AI 玩家
    """

    def __init__(self, color, max_depth):
        """
        玩家初始化
        :param color: 下棋方，'X' - 黑棋，'O' - 白棋
        """

        self.color = color
        
        self.INFINITY = 100
        
        #tune this hyparameter
        self.max_depth = max_depth
        
        if color == 'X':
            self.color_oppo = 'O'
        else:
            self.color_oppo = 'X'
        
        
    def game_over(self,board):
        b_list = list(board.get_legal_actions('X'))
        w_list = list(board.get_legal_actions('O'))
        is_over =  len(b_list) == 0 and len(w_list) == 0
        return is_over 
    
    def alpha_beta_search(self, board):
        best_action = None
        
        best_action, _ = self.max_value(board, -self.INFINITY, self.INFINITY, 0)
     
        return best_action
        
    def max_value(self, board, alpha, beta, depth):
           
        value = -self.INFINITY
        best_action = None
        
        #terminal state
        if self.game_over(board) or depth == self.max_depth:    
            winner, value = board.get_winner()
            if (winner == 0 and self.color == 'X') or (winner == 1 and self.color == 'O'):
                return best_action,value
            else:
                return best_action,-value
        
        depth += 1
        action_list = list(board.get_legal_actions(self.color))
        
        #can move     
        for action in action_list:
            temp_board = copy.deepcopy(board)
            temp_board._move(action, self.color)
            
            
            _, temp_value = self.min_value(temp_board, alpha, beta, depth)
            if temp_value > value:
                value = temp_value
                best_action = action
                
            if value > beta:
                return best_action, value
            alpha = max(alpha, value)
            
        
        #cannot move
        if len(action_list) == 0 :
            temp_board = copy.deepcopy(board)
            _, value = self.min_value(temp_board, alpha, beta, depth)
        
        return best_action, value
        
        
    def min_value(self, board, alpha, beta, depth):
        
        value = self.INFINITY
        best_action = None
        
        if self.game_over(board) or depth == self.max_depth:         
            winner, value = board.get_winner()
            if (winner == 0 and self.color == 'X') or (winner == 1 and self.color == 'O'):
                return best_action, value
            else:
                return best_action, -value
        
        depth += 1
        action_list = list(board.get_legal_actions(self.color_oppo))
        
        #can move
        for action in action_list:
            temp_board = copy.deepcopy(board)
            temp_board._move(action, self.color_oppo)
            
            
            _, temp_value = self.max_value(temp_board, alpha, beta, depth)
            if value > temp_value:
                value = temp_value
                best_action = action
                
            if value < alpha :
                return best_action, value
            beta = min(beta, value)
            
        
        #cannot move
        if len(action_list) == 0 :
            temp_board = copy.deepcopy(board)
            _, value = self.max_value(temp_board, alpha, beta, depth)
        
        return best_action, value
        
        

    def get_move(self, board):
        """
        根据当前棋盘状态获取最佳落子位置
        :param board: 棋盘
        :return: action 最佳落子位置, e.g. 'A1'
        """
        if self.color == 'X':
            player_name = '黑棋'
        else:
            player_name = '白棋'
        print("请等一会，对方 {}-{} 正在思考中...".format(player_name, self.color))

        # -----------------请实现你的算法代码--------------------------------------
        action = self.alpha_beta_search(board)
        # ------------------------------------------------------------------------

        return action


In [16]:
from treelib import Node, Tree

tree = Tree()

if len(tree) == 0:
    print('1')

tree.create_node(identifier='harry', data=(1,2))

if len(tree) == 0:
    print('3')

tree.create_node(tag='harry',identifier='jane', parent='harry', data=(2,3))
tree.create_node(identifier='bill', parent='harry', data=(3,5))
node = tree.create_node(tag='sfd', parent='jane', data=(1,10))

if len(tree) == 0:
    print('2')

print(tree.parent('bill').identifier)

1
harry


# 2.3 UCT

In [62]:
from copy import deepcopy
from treelib import Tree, Node
import math

class AIPlayer:
    """
    AI 玩家
    """

    def __init__(self, color):
        """
        玩家初始化
        :param color: 下棋方，'X' - 黑棋，'O' - 白棋
        """

        self.color = color
        self.Cp = 0.2
        self.root = 'root'
        self.tree = Tree()
        self.times = 100
        self.max_deep = 15
        
        
    def cal_color(self, color):
        if color == 'X':
            return 'O'
        else:
            return 'X'
        
    def game_over(self,board):
        b_list = list(board.get_legal_actions('X'))
        w_list = list(board.get_legal_actions('O'))
        is_over =  len(b_list) == 0 and len(w_list) == 0
        return is_over 
    
    
    def uct_search(self, board):
        
        new_board = deepcopy(board)
        self.tree.create_node(identifier=self.root, data=(0,0, new_board, self.color))
        
        #explore and exploit
        cnt = 0
        while cnt < self.times:
            temp_node_id = self.tree_policy(self.root)
            winner = self.defautl_policy(temp_node_id)
            self.backup(temp_node_id, winner)
            cnt += 1
        
        #modify tree    
        #without memory version
        
        
        
        #new_root, best_action = best_child(v0, 0)
        best_child_id = self.best_child(self.root, 0)
        best_action = self.tree.get_node(best_child_id).tag
        self.tree = Tree()
        
        return best_action
            
    
    
    def tree_policy(self, node_id):
        
        #initialize
        node = self.tree.get_node(node_id)
        Q,N,board,color = node.data
        action_list = list(board.get_legal_actions(color))
        child_list = self.tree.children(node_id)
        
        while self.game_over(board) != True:
          
            if len(action_list) > len(child_list) or (len(action_list) == 0 and len(child_list) == 0):
                return self.expand(node_id)
            else:
                node_id = self.best_child(node_id, self.Cp)
                
                node = self.tree.get_node(node_id)
                Q,N,board,color = node.data
                action_list = list(board.get_legal_actions(color))
                child_list = self.tree.children(node_id)
        
        return node_id
        
    
    
    def expand(self, node_id):
        
        #initialize
        node = self.tree.get_node(node_id)
        Q,N,board,color = node.data
        action_list = list(board.get_legal_actions(color))
        child_list = self.tree.children(node_id)
        tried_list = []
        
        for child in child_list:
            tried_list.append(child.tag)
        
        action = None
        while len(action_list)>0:
            action = random.choice(action_list)
            if action not in tried_list: break
                
        new_board = deepcopy(board)
        if action != None:
            new_board._move(action, color)
        
        new_node = self.tree.create_node(parent=node_id, tag=action, data=(0,0,new_board,self.cal_color(color)))
        
        return new_node.identifier
        
    
    
    def best_child(self, node_id, c):
        
        best_value = -float('inf')
        best_child = None
        
        _,N_all,_,_ = self.tree.get_node(node_id).data
        child_list = self.tree.children(node_id)
        
        for child in child_list:
            
            Q,N,board,color = child.data
            temp_value = Q/N + c*math.sqrt(2*math.log(N_all)/N)
            if temp_value > best_value:
                best_value = temp_value
                best_child = child
                
        return best_child.identifier
        
    
    def defautl_policy(self, node_id):
        
        node = self.tree.get_node(node_id)
        _,_,board,color = node.data
        
        temp_board = deepcopy(board)
        
        cnt = 0
        while self.game_over(temp_board) != True and cnt<self.max_deep :
            cnt+=1
            action_list = list(temp_board.get_legal_actions(color))
            if len(action_list) != 0:
                action = random.choice(action_list)
                temp_board = deepcopy(temp_board)
                temp_board._move(action, color)
                color = self.cal_color(color)
            else:
                color = self.cal_color(color)
        
            
        winner,_ = temp_board.get_winner()
        return winner
    
    
    def backup(self, node_id, winner):
        
        temp_node_id = node_id
        while True:
            temp_node = self.tree.get_node(temp_node_id)
            Q,N,board,color = temp_node.data
            
            if (winner == 0 and color == 'O') or (winner == 1 and color == 'X'):
                Q += 1  
           
            temp_node.data = (Q, N+1, board, color)
            if temp_node_id == self.root:
                break
            temp_node_id = self.tree.parent(temp_node_id).identifier
        
        
            
        
    

    def get_move(self, board):
        """
        根据当前棋盘状态获取最佳落子位置
        :param board: 棋盘
        :return: action 最佳落子位置, e.g. 'A1'
        """
        if self.color == 'X':
            player_name = '黑棋'
        else:
            player_name = '白棋'
        print("请等一会，对方 {}-{} 正在思考中...".format(player_name, self.color))

        # -----------------请实现你的算法代码--------------------------------------
        action = self.uct_search(board)
        # ------------------------------------------------------------------------

        return action


# 2.4 测试

In [75]:
# 导入黑白棋文件
from game import Game  


# 人类玩家黑棋初始化
black_player =  AIPlayer0("X",5)

# AI 玩家 白棋初始化
white_player = AIPlayer("O")

# 游戏初始化，第一个玩家是黑棋，第二个玩家是白棋
game = Game(black_player, white_player)

# 开始下棋
game.run()


=====开始游戏!=====

  A B C D E F G H
1 . . . . . . . .
2 . . . . . . . .
3 . . . . . . . .
4 . . . O X . . .
5 . . . X O . . .
6 . . . . . . . .
7 . . . . . . . .
8 . . . . . . . .
统计棋局: 棋子总数 / 每一步耗时 / 总时间 
黑   棋: 2 / 0 / 0
白   棋: 2 / 0 / 0

请等一会，对方 黑棋-X 正在思考中...
请等一会，对方 黑棋-X 正在思考中...
  A B C D E F G H
1 . . . . . . . .
2 . . . . . . . .
3 . . . X . . . .
4 . . . X X . . .
5 . . . X O . . .
6 . . . . . . . .
7 . . . . . . . .
8 . . . . . . . .
统计棋局: 棋子总数 / 每一步耗时 / 总时间 
黑   棋: 4 / 0 / 0
白   棋: 1 / 0 / 0

请等一会，对方 白棋-O 正在思考中...
请等一会，对方 白棋-O 正在思考中...
  A B C D E F G H
1 . . . . . . . .
2 . . . . . . . .
3 . . . X . . . .
4 . . . X X . . .
5 . . O O O . . .
6 . . . . . . . .
7 . . . . . . . .
8 . . . . . . . .
统计棋局: 棋子总数 / 每一步耗时 / 总时间 
黑   棋: 3 / 0 / 0
白   棋: 3 / 1 / 1

请等一会，对方 黑棋-X 正在思考中...
请等一会，对方 黑棋-X 正在思考中...
  A B C D E F G H
1 . . . . . . . .
2 . . . . . . . .
3 . . . X . . . .
4 . . . X X . . .
5 . . O O X . . .
6 . . . . X . . .
7 . . . . . . . .
8 . . . . . . . .
统计棋局: 棋子总数 / 每一步耗时 

请等一会，对方 黑棋-X 正在思考中...
  A B C D E F G H
1 O O O O O . . .
2 X X X X O . . .
3 X . X X . O . .
4 X O X X O O . .
5 X X O O X O X .
6 X . . O X X . .
7 . . . . X . X .
8 . . . . X . . X
统计棋局: 棋子总数 / 每一步耗时 / 总时间 
黑   棋: 21 / 14 / 146
白   棋: 14 / 1 / 19

请等一会，对方 白棋-O 正在思考中...
请等一会，对方 白棋-O 正在思考中...
  A B C D E F G H
1 O O O O O . . .
2 O X X X O . . .
3 O . X X . O . .
4 O O X X O O . .
5 O X O O X O X .
6 O . . O X X . .
7 O . . . X . X .
8 . . . . X . . X
统计棋局: 棋子总数 / 每一步耗时 / 总时间 
黑   棋: 16 / 14 / 146
白   棋: 20 / 1 / 20

请等一会，对方 黑棋-X 正在思考中...
请等一会，对方 黑棋-X 正在思考中...
  A B C D E F G H
1 O O O O O . . .
2 O X X X X X . .
3 O . X X . X . .
4 O O X X O X . .
5 O X O O X X X .
6 O . . O X X . .
7 O . . . X . X .
8 . . . . X . . X
统计棋局: 棋子总数 / 每一步耗时 / 总时间 
黑   棋: 21 / 5 / 151
白   棋: 16 / 1 / 20

请等一会，对方 白棋-O 正在思考中...
请等一会，对方 白棋-O 正在思考中...
  A B C D E F G H
1 O O O O O . . .
2 O X X X X X . .
3 O . X X . X . .
4 O O X X O X . .
5 O X O O X O X .
6 O . . O O O O .
7 O . . . X . X .
8 . . . . X . . 